In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

4.1

In [ ]:
I = cv2.imread('I.jpg')
I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
J = cv2.imread('J.jpg')
J = cv2.cvtColor(J, cv2.COLOR_BGR2GRAY)

I_small = cv2.resize(I, (0, 0), fx=0.25, fy=0.25)
J_small = cv2.resize(J, (0, 0), fx=0.25, fy=0.25)

difference = cv2.absdiff(I, J)
plt.subplots(1, 3, figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(I, 'gray')
plt.axis('off')
plt.subplot(1, 3, 2)
plt.imshow(J, 'gray')
plt.axis('off')
plt.subplot(1, 3, 3)
plt.imshow(difference, 'gray')
plt.axis('off')
plt.show()

In [ ]:

def block_optical_flow(I, J, W2=3, dX=3, dY=3):
    u = np.zeros(I.shape).astype(np.float32)
    v = np.zeros(I.shape).astype(np.float32)

    min_dim_required = 2 * W2 + 1
    if I.shape[0] < min_dim_required or I.shape[1] < min_dim_required or J.shape[0] < min_dim_required or J.shape[1] < min_dim_required:
        return u, v

    for j in range(W2, I.shape[0] - W2):
        for i in range(W2, I.shape[1] - W2):
            IO = I[j - W2:j + W2 + 1, i - W2:i + W2 + 1].astype(np.float32)

            best_match_offset_x = 0
            best_match_offset_y = 0
            min_dist = float('inf')

            for l in range(-dY, dY + 1):
                for k in range(-dX, dX + 1):
                    center_j_J = j + l
                    center_i_J = i + k
                    if center_j_J < W2 or center_j_J >= J.shape[0] - W2 or center_i_J < W2 or center_i_J >= J.shape[1] - W2:
                        continue
                    JO = J[center_j_J - W2:center_j_J + W2 + 1, center_i_J - W2:center_i_J + W2 + 1].astype(np.float32)
                    dist = np.sum(np.square(IO - JO))
                    if dist < min_dist:
                        min_dist = dist
                        best_match_offset_x = k
                        best_match_offset_y = l
                u[j, i] = best_match_offset_x
                v[j, i] = best_match_offset_y
    return u, v
def vis_flow(u, v):
    magnitude, angle = cv2.cartToPolar(u.astype(np.float32), v.astype(np.float32))
    magnitude_normalized = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)

    HSV = np.zeros((u.shape[0], u.shape[1], 3), dtype=np.uint8)
    HSV[:,:,0] = (angle * 90 / np.pi).astype(np.uint8)
    HSV[:,:,1] = magnitude_normalized.astype(np.uint8)
    HSV[:,:,2] = 255

    RGB = cv2.cvtColor(HSV, cv2.COLOR_HSV2RGB)

    plt.imshow(RGB)
    plt.axis('off')
    plt.show()
W2 = dX = dY = 5
u, v = block_optical_flow(I, J, W2=5, dX=5, dY=5)

vis_flow(u, v)

I2 = cv2.imread('cm1.png')
I2 = cv2.cvtColor(I2, cv2.COLOR_BGR2GRAY)
J2 = cv2.imread('cm2.png')
J2 = cv2.cvtColor(J2, cv2.COLOR_BGR2GRAY)
W2 = dX = dY = 3
u2, v2 = block_optical_flow(I2, J2, W2, dX, dY)

vis_flow(u2, v2)

4.2

In [ ]:
def pyramid(im, max_scale):
    images = [im]
    for k in range(1, max_scale):
        prev_im = images[k - 1]
        if prev_im.shape[0] < 2 or prev_im.shape[1] < 2:
            break 
        images.append(cv2.resize(prev_im, (0, 0), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA))
    return images
def pyramid_optical_flow(I_orig, J_orig, W2, dX, dY, num_levels): 

    pyramid_I = pyramid(I_orig, num_levels) 
    pyramid_J = pyramid(J_orig, num_levels)

    actual_num_levels = len(pyramid_I)
    if actual_num_levels == 0:
        return np.zeros_like(I_orig, dtype=np.float32), np.zeros_like(I_orig, dtype=np.float32)

    if actual_num_levels == 1:
        return block_optical_flow(I_orig, J_orig, W2, dX, dY)

    current_I_level = pyramid_I[actual_num_levels - 1]
    current_J_level = pyramid_J[actual_num_levels - 1]

    u_k, v_k = block_optical_flow(current_I_level, current_J_level, W2, dX, dY)


    for level in range(actual_num_levels - 2, -1, -1):
        I_L = pyramid_I[level]
        J_L = pyramid_J[level]

        h_L, w_L = I_L.shape

        u_upsampled = cv2.resize(u_k, (w_L, h_L), interpolation=cv2.INTER_LINEAR) * 2.0
        v_upsampled = cv2.resize(v_k, (w_L, h_L), interpolation=cv2.INTER_LINEAR) * 2.0

        grid_cols, grid_rows = np.meshgrid(np.arange(w_L, dtype=np.float32), np.arange(h_L, dtype=np.float32))

        map_for_remap_x = grid_cols - u_upsampled
        map_for_remap_y = grid_rows - v_upsampled

        J_L_warped = cv2.remap(J_L.astype(np.float32), map_for_remap_x, map_for_remap_y, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)

        dX_residual = dX
        dY_residual = dY

        du_res, dv_res = block_optical_flow(I_L, J_L_warped, W2, dX_residual, dY_residual)

        u_k = u_upsampled + du_res
        v_k = v_upsampled + dv_res

        return u_k, v_k

u3, v3 = pyramid_optical_flow(I, J, W2=5, dX=5, dY=5, num_levels=2)

vis_flow(u3, v3)
#u4, v4 = pyramid_optical_flow(I2, J2, W2=5, dX=5, dY=5, num_levels=2)

#vis_flow(u4, v4)
